In [1]:
import csv
import pandas as pd
import sqlite3
import os

In [2]:
#Loading CSV Files
box_office_data = pd.read_csv("bom.movie_gross.csv.gz")

In [13]:
#Loading SQLite Database
conn = sqlite3.connect("im.db")

In [4]:
box_office_data.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [14]:
table_list_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(table_list_query, conn)

print("Available tables:", tables)

Available tables:             name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [15]:
movie_ratings = pd.read_sql("""SELECT * FROM movie_ratings;""", conn)
movie_basics = pd.read_sql("""SELECT * FROM movie_basics;""", conn)

conn.close()

print(movie_ratings.head())
print(movie_basics.head())

     movie_id  averagerating  numvotes
0  tt10356526            8.3        31
1  tt10384606            8.9       559
2   tt1042974            6.4        20
3   tt1043726            4.2     50352
4   tt1060240            6.5        21
    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80

In [16]:
imdb_data = pd.merge(movie_basics, movie_ratings, on='movie_id', how='left')

print(imdb_data.head())

    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  averagerating  numvotes  
0        2013            175.0    Action,Crime,Drama            7.0      77.0  
1        2019            114.0       Biography,Drama            7.2      43.0  
2        2018            122.0                 Drama            6.9    4517.0  
3        2018              NaN          Comedy,Drama            6.1      13.0  
4        2017             80.0  Comedy,Drama,Fantasy            6.5     119.0  


In [17]:
merged_data = pd.merge(box_office_data, imdb_data, left_on='title', right_on='primary_title', how='left')
print(merged_data)

                                            title      studio  domestic_gross  \
0                                     Toy Story 3          BV     415000000.0   
1                      Alice in Wonderland (2010)          BV     334200000.0   
2     Harry Potter and the Deathly Hallows Part 1          WB     296000000.0   
3                                       Inception          WB     292600000.0   
4                             Shrek Forever After        P/DW     238700000.0   
...                                           ...         ...             ...   
4142                                    The Quake       Magn.          6200.0   
4143                  Edward II (2018 re-release)          FM          4800.0   
4144                                     El Pacto        Sony          2500.0   
4145                                     The Swan  Synergetic          2400.0   
4146                            An Actor Prepares       Grav.          1700.0   

     foreign_gross  year   